In [14]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import requests
from urllib.parse import urlencode

In [15]:
sns.set(
    font_scale=1,
    style="whitegrid",
    rc={'figure.figsize':(12,7)}
        )

# EDA

In [19]:
base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
public_key_customers = 'https://disk.yandex.ru/d/QXrVDUoQvFhIcg'

# Получаем загрузочную ссылку
final_url_customers = base_url + urlencode(dict(public_key_customers=public_key))
response_customers = requests.get(final_url)
download_url_customers = response.json()['href']

# Считываем файл в виде датафрейма
customers = pd.read_csv(download_url_customers)

In [20]:
customers.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [21]:
customers.shape

(99441, 5)

In [22]:
customers.isna().sum()

customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64

In [23]:
public_key_orders = 'https://disk.yandex.ru/d/0zWmh1bYP6REbw'

# Получаем загрузочную ссылку
final_url_orders = base_url + urlencode(dict(public_key_orders=public_key))
response_orders = requests.get(final_url)
download_url_orders = response.json()['href']

# Считываем файл в виде датафрейма
orders = pd.read_csv(download_url_orders)

In [24]:
orders.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [25]:
orders.shape

(99441, 5)

In [26]:
orders.isna().sum()

customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64

In [27]:
orders.order_status.unique()

AttributeError: 'DataFrame' object has no attribute 'order_status'

In [ ]:
orders.head()

In [ ]:
orders.order_status.value_counts()

In [ ]:
# Сразу переведем столбцы с датой в datetime

orders.order_purchase_timestamp = pd.to_datetime(orders.order_purchase_timestamp)
orders.order_approved_at = pd.to_datetime(orders.order_approved_at)
orders.order_delivered_carrier_date = pd.to_datetime(orders.order_delivered_carrier_date)
orders.order_delivered_customer_date = pd.to_datetime(orders.order_delivered_customer_date)
orders.order_estimated_delivery_date = pd.to_datetime(orders.order_estimated_delivery_date)

In [ ]:
today = orders['order_purchase_timestamp'].max() + timedelta(days=1)

In [ ]:
today

In [ ]:
# Посмотрим сколько у нас заказов статуса "shipped", которые опаздывают

orders.query('order_status == "shipped"')[orders['order_estimated_delivery_date'] < today].shape[0]

In [ ]:
# Насколько заказ опаздывает (в днях), если он не был доставлен, и насколько опоздал, если уже доставлен?

orders['is_late'] = np.where(orders['order_status'] == 'delivered', (orders['order_delivered_customer_date'] - orders['order_estimated_delivery_date']).dt.days, (today - orders['order_estimated_delivery_date']).dt.days)
orders['is_late'] = np.where(orders['is_late'] > 0, orders['is_late'], 0)

In [ ]:
orders.dtypes

In [ ]:
# Примем, что заказ, который имеет статус "shipped" и опаздывает более, чем на 30 дней - не будет доставлен. 
# Посмотрим долю таких заказов, чтобы принять решение, куда определить статус "shipped"

orders.query('order_status == "shipped" & is_late > 30').shape[0] / orders.query('order_status == "shipped"').shape[0]

In [ ]:
# Аналогично посмотрим на заказы со статусом "approved"

orders.query('order_status == "approved" & is_late > 30').shape[0] / orders.query('order_status == "approved"').shape[0]

Таких заказов практически 100%, поэтому можем смело считать, что заказы со статусом "approved" и "shipped" не являются покупками (неизвесто, дойдут они или нет)

# Покупкой будем считать оплаченные заказы, которые, вероятнее всего, будут доставлены, то есть остаются только заказы со статусом "delivered" (100% заказов "approved" и 99.99% заказов "shipped" опаздывают более, чем на месяц, и их можно не учитывать).

# Часть 1

In [ ]:
num_orders_by_customers = orders \
    .merge(customers, how='left', on='customer_id') \
    .query('order_status == "delivered"') \
    .groupby('customer_unique_id', as_index=False) \
    .agg({'order_id': 'nunique'}) \
    .rename(columns={'order_id': 'number_of_orders'})

In [ ]:
num_orders_by_customers.sort_values('number_of_orders', ascending=False)

In [ ]:
num_orders_by_customers.number_of_orders.unique()

In [ ]:
num_customers_with_1_order = num_orders_by_customers.query('number_of_orders == 1').shape[0]

In [ ]:
num_customers_with_1_order

In [ ]:
print(f'Пользователей, которые совершили покупку только один раз - {num_customers_with_1_order}')

# Часть 2

In [ ]:
orders.head()

# Будем считать доставленными заказы, у которых либо статус "delivered", либо "shipped", "invoiced", "processing", "created", "approved" с опозданием не более, чем на 30 дней

In [ ]:
orders['is_delivered'] = \
      ((orders['order_status'] == 'delivered') \
    | ((orders['order_status'] == "shipped") \
    | (orders['order_status'] == "invoiced") \
    | (orders['order_status'] == "processing") \
    | (orders['order_status'] == "created") \
    | (orders['order_status'] == "approved")) \
    & (orders['is_late'] <= 30)) * 1

In [ ]:
orders.query('is_delivered == 0').shape[0]

In [ ]:
orders['start_of_month'] = orders['order_purchase_timestamp'].to_numpy().astype('datetime64[M]')

In [ ]:
num_of_orders_is_not_delivered = orders \
    .groupby('start_of_month', as_index=False) \
    .agg({'is_delivered': lambda x: x.count() - x.sum()}) \
    .rename(columns={'is_delivered': 'is_not_delivered'})

In [ ]:
num_of_orders_is_not_delivered

In [ ]:
print(f'В среднем в месяц не доставляется {num_of_orders_is_not_delivered["is_not_delivered"].mean()} \
заказов по разным причинам')

In [ ]:
(orders['order_purchase_timestamp'].min(), orders['order_purchase_timestamp'].max(), today)

# Посмотрим детально на причины 

In [ ]:
is_not_delivered_orders = orders \
    .query('order_status != "delivered"') \
    .groupby('order_status', as_index=False) \
    .agg({'is_delivered': 'count'}) \
    .rename(columns={'is_delivered': 'is_not_delivered'}) \
    .sort_values('is_not_delivered', ascending=False)

In [ ]:
is_not_delivered_orders

In [ ]:
sns.barplot(data=is_not_delivered_orders, x='is_not_delivered', y='order_status')

# Из диаграммы видим, что чуть меньше половины недоставленных заказов приходится на отмененные и недоступные, ну и большая часть - это заказы, которые либо находятся в пути, либо только созданы, либо оплачены, приняты или в обработке. При этом все эти заказы можно считать недоставленными, т.к. они опаздывают более, чем на 30 дней

# Часть 3

In [ ]:
public_key_items = 'https://disk.yandex.ru/d/xCsQ0FiC-Ue4Lg'

# Получаем загрузочную ссылку
final_url_items = base_url + urlencode(dict(public_key_items=public_key))
response_items = requests.get(final_url)
download_url_items = response.json()['href']

# Считываем файл в виде датафрейма
items = pd.read_csv(download_url_items)

In [ ]:
items.head()

In [ ]:
items.shape

In [ ]:
products_with_orders = orders.merge(items, how='outer', on='order_id')

In [ ]:
products_with_orders

In [ ]:
# Преобразуем дату создания заказа в день недели

products_with_orders['day_of_week'] = products_with_orders['order_purchase_timestamp'].dt.day_name()

In [ ]:
high_sales_day = products_with_orders \
    .query('order_status == "delivered"') \
    .groupby('product_id', as_index=False) \
    .agg({'day_of_week': lambda x: x.mode()})

# Для каждого товара посчитали день (дни), когда его покупали чаще всего

In [ ]:
high_sales_day

# Часть 4

# Как и в шаге 1, отберем только покупки

In [ ]:
num_orders_by_customers_every_week = orders \
    .merge(customers, how='left', on='customer_id') \
    .query('order_status == "delivered"')

In [ ]:
num_orders_by_customers_every_week

In [ ]:
# Считаем сколько недель в месяце

num_orders_by_customers_every_week['weeks_in_month'] = \
    num_orders_by_customers_every_week['order_purchase_timestamp'].dt.days_in_month / 7

In [ ]:
num_orders_by_customers_every_week['start_of_month'] = \
    num_orders_by_customers_every_week['order_purchase_timestamp'].to_numpy().astype('datetime64[M]')

In [ ]:
# Группируем для подсчета количества заказов каждого пользователя за месяц

weekly_sales = num_orders_by_customers_every_week \
    .groupby(['customer_unique_id', 
              'start_of_month',
              'weeks_in_month'], as_index=False) \
    .agg({'order_id': 'count'}) \
    .rename(columns={'order_id': 'number_of_orders_per_month'})

In [ ]:
weekly_sales

In [ ]:
weekly_sales.number_of_orders_per_month.unique()

In [ ]:
# Получаем среднее количество заказов каждым пользователем в неделю по месяцам

weekly_sales['avg_orders'] = weekly_sales['number_of_orders_per_month'] / weekly_sales['weeks_in_month']

In [ ]:
weekly_sales.sort_values('start_of_month')

In [ ]:
np.sort(weekly_sales.avg_orders.unique())

# Часть 5 
# Когортный анализ пользователей по оценке retention

In [ ]:
# Табличка с заказами и выручкой от каждого из них

orders_with_revenue = products_with_orders \
    .groupby('order_id', as_index=False) \
    .agg({'price': 'sum'}) \
    .rename(columns={'price': 'revenue'}) 

In [ ]:
total_table = customers.merge(orders, how='left', on='customer_id').merge(orders_with_revenue, how='left', on='order_id').query('order_status == "delivered"')
total_table

In [ ]:
users = total_table

In [ ]:
users.shape

In [ ]:
# Для каждого пользователя считаю дату регистрации

reg_dates = users \
    .groupby('customer_unique_id', as_index=False) \
    .agg({'order_purchase_timestamp': 'min'}) \
    .rename(columns={'order_purchase_timestamp': 'reg_date'})
reg_dates

In [ ]:
# Вношу эту дату регистрации в табличку users

users = users.merge(reg_dates, how='left', on='customer_unique_id')

In [ ]:
users = users.rename(columns={'start_of_month': 'transaction_month'})

In [ ]:
# Т.к. для когортного анализа меня интересует месяц регистрации, а не дата целиком, преобразую reg_date в reg_month

users['reg_month'] = users['reg_date'].to_numpy().astype('datetime64[M]')

In [ ]:
users = users[['customer_unique_id', 'reg_month', 'transaction_month']]

In [ ]:
users.head()

In [ ]:
users.transaction_month.unique()

In [ ]:
# Далее мне понадобится cross-join для того, чтобы каждому пользователю сопостовлять каждый месяц с начала его регистрации, 
# и считать его активность помесячно

min_date = users.reg_month.min()
max_date = users.reg_month.max()

In [ ]:
# Для этого предварительно создадим отдельную табличку со всеми месяцами

months = pd.DataFrame(pd.date_range(min_date, max_date, freq='MS'))

In [ ]:
months.columns = ['month']

In [ ]:
months['month'] = pd.to_datetime(months['month'])
months

In [ ]:
months.dtypes

In [ ]:
months.shape

In [ ]:
# Cross-join по колонке, состоящей из одинакового символа

users['key'] = 0
months['key'] = 0

In [ ]:
table = users.merge(months, how='outer', on='key')

In [ ]:
table.dtypes

In [ ]:
table['month'] = pd.to_datetime(table['month'])

In [ ]:
# Теперь отдельно создадим табличку с транзакциями по каждому пользователю помесячно, 
# чтобы в дальнейшем всё это соединить в одну табличку

transactions = total_table.groupby(['customer_unique_id', 'start_of_month'], as_index=False).agg({'revenue': 'sum'})

In [ ]:
transactions = transactions.rename(columns={'start_of_month': 'transaction_month'})

In [ ]:
transactions

In [ ]:
# Наконец, создаем финальную табличку по каждому пользователю, в которой каждому пользователю соответсвуют каждый месяц

table = table.merge(transactions, how='left', \
            left_on=['customer_unique_id', 'month'], \
            right_on=['customer_unique_id', 'transaction_month']) \
    .drop(columns=['key', 'transaction_month_x', 'transaction_month_y'])

In [ ]:
# Нас не интересуют месяцы, которые были раньше месяца регистрации пользователя, поэтому просто удалим эти строки

table = table[table['month'] >= table['reg_month']]

In [ ]:
# Для подсчета retention'a создадим кононку 'action', которая будет равна 1, если пользователь был активен в этом месяце,
# и 0 в ином случае

table['action'] = (table['revenue'] > 0) * 1

In [ ]:
table

In [ ]:
# Для проверки посчитаю сколько у меня получилось уникальных пользователей, которые совершили покупку

table.query('action == 1').groupby('customer_unique_id', as_index=False).agg({'action': 'sum'}).shape[0]

In [ ]:
# И для исходной таблички

total_table.groupby('customer_unique_id', as_index=False).agg({'order_id': 'nunique'}).shape[0]

In [ ]:
# Для построения когортной таблицы мне понадобится колонка "life-time", то есть просто число месяцев после регистрации

table['life_time'] = (table.month.dt.to_period('M') - table.reg_month.dt.to_period('M')).apply(lambda x: x.n)

In [ ]:
table

In [ ]:
# И теперь можем построить эту самую когортную табличку retention'a 

retention = table.pivot_table(index='reg_month', columns='life_time', values='action', aggfunc='mean')

In [ ]:
retention

In [ ]:
# Сделаем ее более удобочитаемой

ur_style = (retention
            .style
            .set_caption('User retention by cohort')  # добавляем подпись
            .background_gradient(cmap='viridis')  # раскрашиваем ячейки по столбцам
            .highlight_null('white')  # делаем белый фон для значений NaN
            .format("{:.2%}", na_rep=""))  # числа форматируем как проценты, NaN заменяем на пустоту
ur_style

# Когорта с самым высоким retention на 3-й месяц - это когорта 2017-06, у которой этот показатель равняется 1.3%

In [ ]:
print(f'Опоздавших заказов {total_table[total_table["is_late"] > 0].shape[0] / total_table.shape[0] * 100:.2f}%')

# Из когортного анализа можем сделать вывод, что пользователи имеют очень низкий retention, то есть имеет место быть очень сильный отток, на вопрос почему я затрудняюсь ответить ввиду недостаточного количества данных, т.к. мое предположение о некачественной доставке, скорее всего, неверно (всего около 7% заказов, доставленных позже срока)

# Часть 6
# RFM-segmentation

In [ ]:
rfm_table = total_table.groupby('customer_unique_id', as_index=False).agg({'order_purchase_timestamp': 'max',
                                                               'order_id': 'nunique',
                                                               'revenue': 'sum'}) \
    .rename(columns={'order_purchase_timestamp': 'recency',
                     'order_id': 'frequency',
                     'revenue': 'monetary'})

In [ ]:
rfm_table

In [ ]:
rfm_table['recency'] = (today - rfm_table['recency']).dt.days

# Для создания R-кластера я воспользуюсь разбивкой на квантили с помощью функции pd.qcut() с параметром labels=False для получания не диапазонов, а целых значений. Далее, инвертирую значения, чтобы R и recency находились в обратной пропорции: чем выше recency, тем ниже R-score.

In [ ]:
rfm_table['r'] = (4 - pd.qcut(rfm_table['recency'], q=4, labels=False))

In [ ]:
rfm_table

In [ ]:
rfm_table.groupby('r', as_index=False).agg({'recency': ['mean', 'count']})

In [ ]:
# rfm_table['f'] = (4 - pd.qcut(rfm_table['frequency'], q=4, labels=False, duplicates='drop'))

In [ ]:
# rfm_table.groupby('f', as_index=False).agg({'frequency': ['mean', 'count']})

In [ ]:
rfm_table.frequency.value_counts()

# Понимаем, что с помощью квантилей не получится корректно разбить наш столбец frequency на бакеты, т.к. в нем слишком много "1" и "2". Сделаем F-кластер вручную с помощью функции f(x), предварительно посмотрев на данные частот

In [ ]:
def f(x):
    if x == 1:
        return 1
    if x in [2, 3]:
        return 2
    if x in [4, 5, 6]:
        return 3
    return 4

In [ ]:
rfm_table['f'] = rfm_table['frequency'].apply(f)

In [ ]:
rfm_table

In [ ]:
rfm_table.groupby('f', as_index=False).agg({'frequency': ['mean', 'count']})

# M-кластер попробуем создать через квантили

In [ ]:
rfm_table['m'] = pd.qcut(rfm_table['monetary'], q=4, labels=False) + 1

In [ ]:
rfm_table

In [ ]:
rfm_table.groupby('m', as_index=False).agg({'monetary': ['mean', 'count']})

In [ ]:
rfm_table['monetary'].min(), rfm_table['monetary'].max()

# С помощью квантилей кластеризация явно некорректная, поэтому создам его вручную для более правильного разбиения

In [ ]:
sns.displot(rfm_table.monetary, height=8.27, aspect=11.7/8.27)

In [ ]:
# Посмотрим в масштабе подробнее правую часть распределения

sns.displot(rfm_table.query('monetary > 1000').monetary, height=8.27, aspect=11.7/8.27)

In [ ]:
def m(x):
    if 0 <= x < 400:
        return 1
    if 400 <= x < 1200:
        return 2
    if 1200 <= x < 4000:
        return 3
    return 4

In [ ]:
rfm_table['m'] = rfm_table['monetary'].apply(m)

In [ ]:
rfm_table.groupby('m', as_index=False).agg({'monetary': ['mean', 'count']})

In [ ]:
rfm_table['rfm_score'] = rfm_table['r'] + rfm_table['f'] + rfm_table['m']

In [ ]:
rfm_table['rfm_segment'] = (rfm_table['r']).astype(str) + (rfm_table['f']).astype(str) + (rfm_table['m']).astype(str)

In [ ]:
rfm_table

In [ ]:
rfm_table \
    .groupby('rfm_score', as_index=False) \
    .agg({'customer_unique_id': 'nunique'}) \
    .rename(columns={'customer_unique_id': 'number_of_customers'}) \
    .sort_values('number_of_customers', ascending=False) \
    .head(10)

# Как видно из таблички выше, большая часть пользователей имеет очень низкий RFM-score. Посмотрим чуть подробнее

In [ ]:
# Посмотрим какие кластеры включают в себя наибольшее количество пользователей

size_of_segments = rfm_table \
    .groupby('rfm_segment', as_index=False) \
    .agg({'customer_unique_id': 'nunique'}) \
    .rename(columns={'customer_unique_id': 'number_of_customers'}) \
    .sort_values('number_of_customers', ascending=False)
size_of_segments.head(10)

In [ ]:
# Посмотрим как распределены сегменты

ax = sns.displot(rfm_table.rfm_segment, height=8.27, aspect=11.7/8.27).set_xticklabels(rotation=45)

In [ ]:
rfm = total_table.groupby('customer_unique_id', as_index=False).agg({'order_purchase_timestamp': 'max',
                                                               'order_id': 'nunique',
                                                               'revenue': 'sum'}) \
    .rename(columns={'order_purchase_timestamp': 'recency',
                     'order_id': 'frequency',
                     'revenue': 'monetary'})

In [ ]:
rfm['recency'] = (today - rfm['recency']).dt.days

In [ ]:
rfm['recency'] = pd.qcut(rfm['recency'], q=4)

In [ ]:
r = rfm['recency'].drop_duplicates().sort_values(ascending=False).reset_index(drop=True)

In [ ]:
f = pd.Series(data=([0, 1], [2, 3], [4, 5, 6], [7, 17]))

In [ ]:
m = pd.Series(data=([0, 399.9], [400, 1199.9], [1200, 3999.9], [4000, 13440]))

In [ ]:
rfm = pd.DataFrame(data={'score': range(1, 5), 'recency': r, 'frequency': f, 'monetary': m})

In [ ]:
rfm_segments = rfm_table[['rfm_segment', 'r', 'f', 'm']].merge(rfm[['score', 'recency']], how='left', left_on='r', right_on='score')
rfm_segments = rfm_segments.merge(rfm[['score', 'frequency']], how='left', left_on='f', right_on='score')
rfm_segments = rfm_segments.merge(rfm[['score', 'monetary']], how='left', left_on='m', right_on='score')

In [ ]:
rfm_segments = rfm_segments[['rfm_segment', 'recency', 'frequency', 'monetary']] \
    .drop_duplicates('rfm_segment').reset_index(drop=True)

In [ ]:
rfm_segments['recency'] = rfm_segments['recency'].astype(str) + ' дней'
rfm_segments['frequency'] = rfm_segments['frequency'].astype(str) + ' заказов'
rfm_segments['monetary'] = rfm_segments['monetary'].astype(str) + ' рублей'

In [ ]:
rfm_segments.merge(size_of_segments, how='inner', on='rfm_segment')

Выше табличка имеет все сегменты, которые удалось получить из наших данных. Столбцы R, F, M заполнены диапазонами значений соответсвующего сегмента и количество клиентов в каждом из сегментов

# Из распределения сегментов можно сделать вывод о том, практически все пользователи находятся в кластерах 411, 111, 211, 311. То есть это люди, которые сделали в разное время всего 1 заказ на небольшую сумму (до 400 рублей). Это еще раз подтверждает вывод, который я сделал в результате когортного анализа о высоком оттоке по разным причинам. Низкий чек заказов не подзволяет говорить о том, что такой низкий retention- это просто специфика данного бизнеса (как например сервис по бронированию авиабилетов, в котором пользователи достаточно редко совершают какие-то сделки), поэтому дал бы совет бизнесу либо пересмотреть качество продукции, либо отделу маркетинга, к примеру, с помощью промо-акции попытаться увеличить retention